# Compressed Attribute

This feature can automatically compress the data before it is sent to the DynamoDB.

NOTE: this solution is based on [pynamodb_mate](https://github.com/MacHu-GWU/pynamodb_mate-project) Python library.

**Summary**

Sometimes you want to compress the data before store to save DB space. For example, in an E-commerce data model, an order has many items like this: ``[{"item_name": "apple", "item_count": 12}, {"item_name": "banana", "item_count": 5}]``. There are lots of repeated information such as the keys ``"item_name"`` and ``"item_count"``.

**How it works**

Internally it always serializes your data into binary, and compress it, then send to DynamoDB.

## Define attribute to use Auto Compressed

In [2]:
from rich import print as rprint
import pynamodb_mate as pm

# Define the Data Model to use compressed attribute
class OrderModel(pm.Model):
    class Meta:
        table_name = f"pynamodb-mate-example-compress"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    order_id = pm.UnicodeAttribute(hash_key=True)

    # original value is unicode str
    description = pm.CompressedUnicodeAttribute(null=True)

    # original value is binary bytes
    image = pm.CompressedBinaryAttribute(null=True)

    # original value is any json serializable object
    items = pm.CompressedJSONDictAttribute(null=True)

OrderModel.create_table(wait=True)

## Write and Read the Item

In [3]:
# Create an item
order_id = "order_001"
description = "a fancy order!" * 10
image = description.encode("utf-8") # a fake binary object
items = [
    {
        "item_id": "i_001",
        "item_name": "apple",
        "item_price": 2.4,
        "quantity": 8,
    },
    {
        "item_id": "i_002",
        "item_name": "banana",
        "item_price": 0.53,
        "quantity": 5,
    },
]
order = OrderModel(
    order_id=order_id,
    description=description,
    image=image,
    items=items,
)
# Save item to DynamoDB
order.save()
print(f"preview the DynamoDB item: {order.item_detail_console_url}")
print("you will see that the raw data in DynamoDB is compressed")

preview the DynamoDB item: https://us-east-1.console.aws.amazon.com/dynamodbv2/home?region=us-east-1#edit-item?table=pynamodb-mate-example-compress&itemMode=2&pk=order_001&sk&ref=%23item-explorer%3Ftable%3Dpynamodb-mate-example-compress&route=ROUTE_ITEM_EXPLORER
you will see that the raw data in DynamoDB is compressed


In [4]:
# Get the value back and verify
order = OrderModel.get(order_id)
rprint(order.to_dict())

{
    'description': 'a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy 
order!a fancy order!a fancy order!a fancy order!',
    'image': b'a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a fancy order!a 
fancy order!a fancy order!a fancy order!',
    'items': [
        {'item_id': 'i_001', 'item_name': 'apple', 'item_price': 2.4, 'quantity': 8},
        {'item_id': 'i_002', 'item_name': 'banana', 'item_price': 0.53, 'quantity': 5}
    ],
    'order_id': 'order_001'
}